# the `BusyFlyerMixin`

In [1]:
import enum
import logging
import threading
import time

import ophyd
import bluesky
import bluesky.plans
import databroker

logger = logging.getLogger()
RE = bluesky.RunEngine({})
db = databroker.Broker.from_config(databroker.temp_config())
RE.subscribe(db.insert)

0

In [2]:
BUSY_PV = 'prj:mybusy'
TIME_WAVE_PV = 'prj:t_array'
X_WAVE_PV = 'prj:x_array'
Y_WAVE_PV = 'prj:y_array'

In [3]:
class BusyStatus(str, enum.Enum):
    busy = "Busy"
    done = "Done"

class BusyRecord(ophyd.Device):
    """a busy record sets the fly scan into action"""
    state = ophyd.Component(ophyd.EpicsSignal, "", string=True)
    output_link = ophyd.Component(ophyd.EpicsSignal, ".OUT")
    forward_link = ophyd.Component(ophyd.EpicsSignal, ".FLNK")

class MyWaveform(ophyd.Device):
    """waveform records store fly scan data"""
    wave = ophyd.Component(ophyd.EpicsSignalRO, "")
    number_elements = ophyd.Component(ophyd.EpicsSignalRO, ".NELM")
    number_read = ophyd.Component(ophyd.EpicsSignalRO, ".NORD")

In [ ]:
class BusyFlyerMixin(object):
    """
    support APS Fly Scans that are operated by a busy record
    """

    def __init__(self, **kwargs):
        self._flyscan_status = None
        self.poll_sleep_interval_s = 0.05
        
        #FIXME:  
        #try:
        #    if not isinstance(self.busy, BusyRecord):
        #        msg = "``busy`` must be a ``BusyRecord`` instance"
        #        raise KeyError(msg)
        #except AttributeError:
        #    msg = "must define ``busy`` record instance"
        #    raise KeyError(msg)
    
    def hook_flyscan(self):
        """
        Customize: called during fly scan
        
        called from RunEngine thread in ``flyscan_plan()``, 
        blocking calls are not permitted
        """
        logger.debug("hook_flyscan_not_scanning() : no-op default")
    
    def hook_pre_flyscan(self):
        """
        Customize: called before the fly scan
        
        NOTE: As part of a BlueSky plan thread, no blocking calls are permitted
        """
        logger.debug("hook_pre_flyscan() : no-op default")
    
    def hook_post_flyscan(self):
        """
        Customize: called after the fly scan
        
        NOTE: As part of a BlueSky plan thread, no blocking calls are permitted
        """
        logger.debug("hook_post_flyscan() : no-op default")
    
    def hook_flyscan_wait_not_scanning(self):
        """
        Customize: called ``flyscan_wait(False)``
        
        called in separate thread, blocking calls are permitted
        but keep it quick
        """
        logger.debug("hook_flyscan_not_scanning() : no-op default")

    def hook_flyscan_wait_scanning(self):
        """
        Customize: called from ``flyscan_wait(True)``
        
        called in separate thread, blocking calls are permitted
        but keep it quick
        """
        logger.debug("hook_flyscan_wait_scanning() : no-op default")

    def flyscan_wait(self, scanning):
        """
        wait for the busy record to return to Done
        
        Call external hook functions to allow customizations
        """
        msg = "flyscan_wait()"
        msg += " scanning=" + str(scanning)
        msg += " busy=" + str(self.busy.state.value)
        logger.debug(msg)

        if scanning:
            hook = self.hook_flyscan_wait_scanning
        else:
            hook = self.hook_flyscan_wait_not_scanning

        while self.busy.state.value not in (BusyStatus.done, 0):
            hook()
            time.sleep(self.poll_sleep_interval_s)  # wait to complete ...

    def _flyscan(self):
        """
        start the busy record and poll for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.debug("_flyscan()")
        if self._flyscan_status is None:
            logger.debug("leaving fly_scan() - not complete")
            return

        logger.debug("flyscan() - clearing Busy")
        self.busy.state.put(BusyStatus.done) # make sure it's Done first
        self.flyscan_wait(False)
        time.sleep(1.0)

        logger.debug("flyscan() - setting Busy")
        self.busy.state.put(BusyStatus.busy)
        self.flyscan_wait(True)

        self._flyscan_status._finished(success=True)
        logger.debug("flyscan() complete")
    
    def flyscan_plan(self, *args, **kwargs):
        """
        This is the BlueSky plan to submit to the RunEgine
        """
        logger.debug("flyscan_plan()")
        yield from bpp.open_run()

        self.hook_pre_flyscan()
        self._flyscan_status = DeviceStatus(self.busy.state)
        
        thread = threading.Thread(target=self._flyscan, daemon=True)
        thread.start()
        
        while not self._flyscan_status.done:
            self.hook_flyscan()
            bps.sleep(self.poll_sleep_interval_s)
        logger.debug("plan() status=" + str(self._flyscan_status))
        self.hook_post_flyscan()

        yield from bpp.close_run()
        logger.debug("plan() complete")
